In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

In [46]:
# import momepy as mm

In [2]:
# https://ipython-books.github.io/147-creating-a-route-planner-for-a-road-network/

In [2]:
gdf_big = gpd.read_file('./total/gdf_line_all_n_samobl.json',
                        encoding='utf-8')

In [3]:
gdf_left = gpd.read_file('./total/gdf_line_sam_tol_new2.json',
                        encoding='utf-8')

In [4]:
gdf_all = gdf_big.append(gdf_left)
gdf_all = gdf_all.drop_duplicates(subset=['line_id', 
                                          'trip_id']).reset_index(drop=True)

In [5]:
stps_big = pd.read_csv('./total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [6]:
reestr_big = pd.read_csv('./total/df_reestr_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [7]:
graph = gpd.read_file('./with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [8]:
nodes = gpd.read_file('./with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [9]:
def get_line_graph(line, graph, nodes, route_type, buffer):
    tmp_grph = graph[((graph.type_ped == 'car_n_ped') 
                     & (graph.lanes != 0))]
    if route_type == 'tramway':
        tmp_grph = tmp_grph[((tmp_grph.type_ts == 'TM,CAR,BUS,TB,MT') 
                      | (tmp_grph.type_ts == 'TM'))].reset_index(drop=True)
    elif route_type == 'all':
        tmp_grph = tmp_grph
    else:
        tmp_grph = tmp_grph[tmp_grph.type_ts != 'TM'].reset_index(drop=True)
    # 
    buff_one_line = gpd.GeoDataFrame(geometry=[line])
    buff_one_line.crs = graph.crs
    buff_one_line.geometry = buff_one_line.geometry.to_crs('epsg:32637').buffer(buffer).to_crs('epsg:4326')
    
    
    line_graph = gpd.sjoin(tmp_grph, buff_one_line[['geometry']], how='inner', op='within')
    line_graph = line_graph.drop('index_right', axis=1).reset_index(drop=True)
    
    wider_graph = gpd.sjoin(tmp_grph, buff_one_line[['geometry']], how='inner', op='intersects')
    wider_graph = wider_graph.drop('index_right', axis=1).reset_index(drop=True)
    
    lst_nodes = list(set(list(line_graph.from_node.unique()) 
                         + list(line_graph.to_node.unique())))
    #
    line_nodes = nodes[nodes.nodeID.isin(lst_nodes)]
        
    return line_graph, line_nodes, wider_graph

In [10]:
def get_closest_pt(ope_pt, one_grph, nodes):
    
    text_trap = io.StringIO()
    with redirect_stdout(text_trap):
        closest_link_id = mm.get_network_id(ope_pt, one_grph, 'link_id')[0]
#     closest_link_id = mm.get_network_id(ope_pt, one_grph, 'link_id')[0]
    link_in_graph = one_grph[one_grph.link_id == closest_link_id].reset_index(drop=True)
    to_node_id = link_in_graph.to_node[0]
    from_node_id = link_in_graph.from_node[0]
    to_node = nodes[nodes.nodeID == to_node_id].reset_index(drop=True)['geometry'][0]
    from_node = nodes[nodes.nodeID == from_node_id].reset_index(drop=True)['geometry'][0]

    dist1 = ope_pt.geometry[0].distance(to_node)
    dist2 = ope_pt.geometry[0].distance(from_node)

    if dist1 < dist2:
        closest_node = to_node
        closest_node_id = to_node_id
    else:
        closest_node = from_node
        closest_node_id = from_node_id
    # 
    return closest_node,closest_node_id

In [11]:
def create_lst_coords_from_df(nodes):
    lst_geo = list(nodes.geometry)
    lst_ndid = list(nodes.nodeID)

    lst_coords=[]
    i=0
    for i in range(len(nodes)):
        lst_coords.append(lst_geo[i].coords[0])
    # 
    return lst_coords

In [12]:
def get_nodeid(res_lst, cnt, lst_coords, lst_ndid):
    segm_seq=[]
    i=0
    for i in range(len(res_lst)):
        one_lst_pt=[]
        one_coord = res_lst[i]
        j=0
        if one_coord in lst_coords:
            ind = lst_coords.index(one_coord)
            nodid = lst_ndid[ind]
        else:
            print(i)
            nodid = 0
        #
        segm_seq.append([cnt, nodid, one_coord])
        cnt+=1
    # 
    return segm_seq, cnt

In [13]:
def get_one_segm_seq(cnt, segm, line_graph, line_nodes):

    ope_pt = gpd.GeoDataFrame(geometry=[Point(segm.coords[0])])
    two_pt = gpd.GeoDataFrame(geometry=[Point(segm.coords[-1])])

    closest_node,closest_node_id = get_closest_pt(ope_pt, line_graph, line_nodes)
    closest_node2,closest_node_id2 = get_closest_pt(two_pt, line_graph, line_nodes)

    one_G = mm.gdf_to_nx(line_graph)
    res_lst = nx.shortest_path(one_G, closest_node.coords[0], closest_node2.coords[0])

    lst_coords = create_lst_coords_from_df(line_nodes)
    lst_ndid = list(line_nodes.nodeID)

    segm_seq, cnt = get_nodeid(res_lst, cnt, lst_coords, lst_ndid)

    return segm_seq, cnt

In [94]:
def get_lst_segm_in_loop(one_segm, lnth_segm):
    cnt_lp = int(lnth_segm / 6)

    if cnt_lp > 1:
        step1=0
        step2 = 6
        
        k=0
        lst_segs=[]
        for k in range(cnt_lp):
            first = one_segm.coords[step1:step2]
            if len(first) >= 2:
                seg_line = LineString(first)
                lst_segs.append(seg_line)
                step1 = step1 + 6
                step2 = step2 + 6
        if step1 < lnth_segm:
            first = one_segm.coords[step1:]
            seg_line = LineString(first)
            lst_segs.append(seg_line)
    else:
        lst_segs = [one_segm]
    #
    return lst_segs

In [92]:
def get_node_seq(line, graph, nodes, route_type, buffer):

#     buffer=7
    # line = one_big_line
    line_graph, line_nodes, wider_graph = get_line_graph(line, graph, nodes, route_type, buffer)

    node_seq = []
    cnt=1
    i=0
    for i in tqdm(range(len(line))):
    # for i in range(2):
        one_segm = line[i]
        lnth_segm = len(one_segm.coords[:])

        lst_segs=[]
        if lnth_segm > 15:
            print(i)
            lst_segs = get_lst_segm_in_loop(one_segm, lnth_segm)
        else:
            lst_segs = [one_segm]
        # 
        l=0
        for l in range(len(lst_segs)):
            segm = lst_segs[l]
        
            try:
                segm_seq, cnt = get_one_segm_seq(cnt, segm, line_graph, line_nodes)
                node_seq = node_seq + segm_seq
            except:
                try:
                    buffer = 50
                    line_graph2, line_nodes2, wider_graph2 = get_line_graph(line, graph, nodes, 'all', buffer)
                    segm_seq, cnt = get_one_segm_seq(cnt, segm, line_graph2, line_nodes2)
                    node_seq = node_seq + segm_seq
                except:
                    cnt+=100
        # 
    node_seq_fin = get_unique_seq(node_seq)
    try:
        node_seq_fin2 = find_first_node(node_seq_fin, wider_graph, line_graph, line)
#         new_node_seq_fin = find_first_last_node(node_seq_fin, wider_graph, line_graph, line)
    except:
        node_seq_fin2 = node_seq_fin
    #
    try:
        new_node_seq_fin = find_last_node(node_seq_fin2, wider_graph, line_graph, line)
    except:
        new_node_seq_fin = node_seq_fin2

    return new_node_seq_fin

In [16]:
def get_unique_seq(node_seq):
    node_seq2 = node_seq[:]

    lst_ind=[]
    j=0
    for j in range(1,len(node_seq2)):
        if node_seq2[j][-1] == node_seq2[j-1][-1]:
            lst_ind.append(node_seq2[j])
    #
    k=0
    for k in range(len(lst_ind)):
        one = lst_ind[k]
        node_seq2.remove(one)
    # 
    node_seq_fin = []
    
    l=0
    for l in range(len(node_seq2)):
        node_seq_fin.append([node_seq2[l][0],node_seq2[l][1],Point(node_seq2[l][2])])
    #
    return node_seq_fin

In [17]:
def get_closest_link(lst_geo, node, lst_links):
    smallest_dist = 100
    i=0
    for i in range(len(lst_geo)):
        if node.distance(lst_geo[i]) < smallest_dist:
            smallest_dist = node.distance(lst_geo[i])
            closest_link = lst_links[i]
    # 
    return closest_link
# 

In [18]:
################## Find inner_angle

def find_inner_angle(line_from, line_to, line_via):

    A = line_from.length #значения слишком маленькие  * 10000
    B = line_to.length
    C = line_via.length
    try:
        angle = math.degrees(math.acos((A * A + B * B - C * C)/(2.0 * A * B)))
    except:
        # if (A + B) <= C:
        try:
            # if (A + B) <= C - its an error, add a little to both
            new_val = ((C - (A + B)) + 0.000002) / 2 
            A = A + new_val
            B = B + new_val
            angle = math.degrees(math.acos((A * A + B * B - C * C)/(2.0 * A * B)))
        except:
            angle = math.degrees(math.atan2(A,B))
#             print("Not ok")
    angle = int(round(angle, 0))
    # 
    return angle

In [19]:
def get_closest_angle(lst_geo, line_to, lst_links):

    lst_angles = []
    i=0
    for i in range(len(lst_geo)):
        line_from = lst_geo[i]
        line_via = LineString([line_to.coords[-1], line_from.coords[0]])
        angle = find_inner_angle(line_from, line_to, line_via)
        lst_angles.append(angle)
    # 

    smallest_angle = 360
    sm_ind=0
    i=0
    for i in range(len(lst_angles)):
        one=lst_angles[i]
        if abs(180-one) < smallest_angle:
            smallest_angle = abs(180-one)
            sm_ind = i
    # 
    smlst_ang_id = lst_links[sm_ind]

    return smlst_ang_id

In [20]:
def find_closest_node(gdf_links, node, second_link, from_to):

    lst_geo = list(gdf_links.geometry)
    lst_links = list(gdf_links.link_id)

    line_to = second_link.geometry[0]

    closest_link = get_closest_link(lst_geo, node, lst_links)

    smlst_ang_id = get_closest_angle(lst_geo, line_to, lst_links)

    while smlst_ang_id != closest_link:
        print("Not equal:", smlst_ang_id, closest_link)
        try:
            ind_not_ok = lst_links.index(closest_link)
            lst_geo_new = lst_geo[:]
            lst_geo_new.remove(lst_geo_new[ind_not_ok])

            lst_lnk_new = lst_links[:]
            lst_lnk_new.remove(closest_link)
            closest_link = get_closest_link(lst_geo_new, node, lst_lnk_new)
        except:
            break
    # 
    if from_to == 'from':
        closest_node = list(gdf_links[gdf_links.link_id == closest_link]['from_node'])[0]
    else:
        closest_node = list(gdf_links[gdf_links.link_id == closest_link]['to_node'])[0]
    #
    
    return closest_node

In [21]:
def get_info_by_node(first_last, node, nodes_gdf, node_seq_fin):

    nodes_coords = list(nodes_gdf[nodes_gdf.nodeID == node]['geometry'])[0]
    new_node_seq_fin=[]
    if first_last == 'first':
        new_node_seq_fin=[[0, node, nodes_coords]]
        new_node_seq_fin = new_node_seq_fin + node_seq_fin
    else:
        seq = node_seq_fin[-1][0] + 1
        new_node_seq_fin=[[seq, node, nodes_coords]]
        new_node_seq_fin = node_seq_fin + new_node_seq_fin
    # 
    return new_node_seq_fin

In [22]:
def find_first_node(node_seq_fin, wider_graph, line_graph, line):

    # first node search
    start_pnt = Point(line[0].coords[0])
    first_id = node_seq_fin[0][1]
    second_id = node_seq_fin[1][1]


    first_links = wider_graph[((wider_graph.to_node == first_id) 
                               & (~wider_graph.link_id.isin(line_graph.link_id)))].reset_index(drop=True)
    second_link = wider_graph[((wider_graph.from_node == first_id) 
                               & (wider_graph.to_node == second_id))].reset_index(drop=True)
    #
    first_closest_node = find_closest_node(first_links, start_pnt, second_link, 'from')


    new_node_seq_fin = get_info_by_node('first', first_closest_node, nodes, node_seq_fin)
    return new_node_seq_fin

In [23]:
def find_last_node(node_seq_fin, wider_graph, line_graph, line):

    # last node search
    end_pnt = Point(line[-1].coords[-1])
    last_id = node_seq_fin[-1][1]
    prelast_id = node_seq_fin[-2][1]

    last_links = wider_graph[((wider_graph.from_node == last_id) 
                              & (~wider_graph.link_id.isin(line_graph.link_id)))].reset_index(drop=True)
    #
    prelast_link = wider_graph[((wider_graph.from_node == prelast_id) 
                               & (wider_graph.to_node == last_id))].reset_index(drop=True)
    #
    last_closest_node = find_closest_node(last_links, end_pnt, prelast_link, 'to')

    new_node_seq_fin = node_seq_fin
    new_node_seq_fin = get_info_by_node('last', last_closest_node, nodes, new_node_seq_fin)
    return new_node_seq_fin

In [24]:
def find_first_last_node(node_seq_fin, wider_graph, line_graph, line):

    # first node search
    start_pnt = Point(line[0].coords[0])
    first_id = node_seq_fin[0][1]
    second_id = node_seq_fin[1][1]


    first_links = wider_graph[((wider_graph.to_node == first_id) 
                               & (~wider_graph.link_id.isin(line_graph.link_id)))].reset_index(drop=True)
    second_link = wider_graph[((wider_graph.from_node == first_id) 
                               & (wider_graph.to_node == second_id))].reset_index(drop=True)
    #
    first_closest_node = find_closest_node(first_links, start_pnt, second_link, 'from')

    ##################
    # last node search
    end_pnt = Point(line[-1].coords[-1])
    last_id = node_seq_fin[-1][1]
    prelast_id = node_seq_fin[-2][1]

    last_links = wider_graph[((wider_graph.from_node == last_id) 
                              & (~wider_graph.link_id.isin(line_graph.link_id)))].reset_index(drop=True)
    #
    prelast_link = wider_graph[((wider_graph.from_node == prelast_id) 
                               & (wider_graph.to_node == last_id))].reset_index(drop=True)
    #
    last_closest_node = find_closest_node(last_links, end_pnt, prelast_link, 'to')

    new_node_seq_fin = get_info_by_node('first', first_closest_node, nodes, node_seq_fin)
    new_node_seq_fin = get_info_by_node('last', last_closest_node, nodes, new_node_seq_fin)

    return new_node_seq_fin

In [54]:
def check_connectivity(one_trip_id, new_node_seq_fin, line, graph, nodes, route_type, buffer):

    # buffer = 20
    line_graph, line_nodes, wider_graph = get_line_graph(line, graph, nodes, route_type, buffer)

    np_seq = np.array(new_node_seq_fin, dtype=object)

    lst_new_seq=[]

    cnt=1
    for i in tqdm(range(1, len(np_seq))):

        segm = LineString([np_seq[i-1,2], np_seq[i,2]])
        try:
            segm_seq, cnt = get_one_segm_seq(cnt, segm, line_graph, line_nodes)
        except:
            cnt+=1
            one_pt = [cnt, np_seq[i-1,1], np_seq[i-1,2].coords[0]]
            two_pt = [cnt+1, np_seq[i,1], np_seq[i,2].coords[0]]
            segm_seq = [one_pt, two_pt]
            print(i)
        #
#         segm_seq = [one_trip_id] + segm_seq
        lst_new_seq = lst_new_seq + segm_seq
    #

    node_seq_fin = get_unique_seq(lst_new_seq)
    i=0
    for i in range(len(node_seq_fin)):
        node_seq_fin[i] = [one_trip_id] + node_seq_fin[i]

    return node_seq_fin

In [135]:
def check_graph(new_seq, np_grph, ind_ndfr, ind_ndto):

    lst_bad = []
    i=0
    for i in range(1, len(new_seq)):
        nd_from = new_seq[i-1][2]
        nd_to = new_seq[i][2]

        one_link = np_grph[(np_grph[:,ind_ndfr] == nd_from) & (np_grph[:,ind_ndto] == nd_to)]
        if len(one_link) == 0:
            lst_bad.append(1)
    # 
    new_seq2 = new_seq[:]
    i=0
    for i in range(len(new_seq)):
        if len(lst_bad) == 0:
            new_seq2[i] = new_seq2[i] + [True]
        else:
            new_seq2[i] = new_seq2[i] + [False]
    #
    return new_seq2
#

In [139]:
buffer=7
buffer2=20

np_ga = gdf_all.to_numpy()
ind_geo = list(gdf_all.columns).index('geometry')
ind_ti = list(gdf_all.columns).index('trip_id')
ind_li = list(gdf_all.columns).index('line_id')


np_rb = reestr_big.to_numpy()
ind_rb_li = list(reestr_big.columns).index('line_id')
ind_rb_tp = list(reestr_big.columns).index('type_ts')


np_grph = graph.to_numpy()
ind_ndfr = list(graph.columns).index('from_node')
ind_ndto = list(graph.columns).index('to_node')
ind_gr_li = list(graph.columns).index('link_id')

In [144]:
big_seq = []
lst_bad = []

i=0
for i in tqdm(range(10)):
# for i in tqdm(range(len(np_ga))):
    line = np_ga[i, ind_geo]
    one_trip_id = np_ga[i, ind_ti]
    one_line_id = int(np_ga[i, ind_li])
    
    route_type = np_rb[np_rb[:,ind_rb_li] == one_line_id][:,ind_rb_tp][0]
    try:
        node_seq_first = get_node_seq(line, graph, nodes, route_type, buffer)
        new_seq = check_connectivity(one_trip_id, node_seq_first, line, 
                                     graph, nodes, route_type, buffer2)
        #
        new_seq = check_graph(new_seq, np_grph, ind_ndfr, ind_ndto)

        big_seq  = big_seq + new_seq
    except:
        lst_bad.append(one_trip_id)
# 

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.92it/s]
C:\Users\ochristm\anaconda3\lib\site-packages\momepy\utils.py:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length
Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 90.91it/s]

6



Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 83.32it/s]

7



Snapping: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.12it/s]

...

4



Snapping: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 100.01it/s]

5



Snapping: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 100.00it/s]


6



In [145]:
print(len(lst_bad))

7


In [141]:
# df_fin = pd.DataFrame(new_seq, columns=['trip_id', 'seq','nodeID','geometry'])
df_fin = pd.DataFrame(big_seq, columns=['trip_id', 'seq','nodeID','geometry', 'is_connected'])
df_fin = df_fin.reset_index(drop=True).reset_index()
df_fin['seq'] = df_fin['index'] + 1
del df_fin['index']
gdf_fin = gpd.GeoDataFrame(df_fin, geometry=df_fin.geometry)
gdf_fin.crs='epsg:4326'

In [98]:
# gdf_fin.to_file('./data/gdf_fin5_new.json', driver='GeoJSON', encoding='utf-8')

In [ ]:
# сделать граф, проверить связность

In [143]:
len(np_ga)

1756